In [48]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf

import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns

import requests
import csv

import warnings

In [49]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [50]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
energy_sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Energy']['Ticker'].tolist()

In [51]:
energy_sector_ticker_list

['APA',
 'BKR',
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL',
 'HES',
 'KMI',
 'MRO',
 'MPC',
 'OXY',
 'OKE',
 'PSX',
 'SLB',
 'TRGP',
 'VLO',
 'WMB']

In [52]:
# # Ask the user to input stock tickers separated by a comma
# tickers_input = input("Enter stock tickers separated by commas:")

# # Split the input string into a list of tickers
# tickers = tickers_input.split(',')

In [53]:
# enable dynamic query

# input_security_dim = input()
# input_security_func = input()

In [77]:
# Parameters section

alpha_vantage_api_key = 'YI2V50P8VRQ3HFKM' # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# Define the ticker symbols as a list; eg. TSM,MSFT,AMZN
# ticker_symbols = input("Enter stock tickers separated by commas:") 
# ticker_symbols = ticker_symbols.split(',')
ticker_symbols = [

    'TSM'
    ,'BABA'
    ,'SYK'

]

# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]


ticker_dict = {}

In [78]:
# Daily quote section
for symbol in ticker_symbols:

    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':

            selected_cols = [
                '4. close'
            ]

            df_stock_base = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            df_stock_base.rename(
                columns={
                    '4. close': f'{symbol}'
                    }
                ,inplace=True
                )
            
            df_stock_base[f'{symbol}'] = df_stock_base[f'{symbol}'].astype(str).apply(lambda x: float(x))
            df_stock_base[f'{symbol}'] = df_stock_base[f'{symbol}'].round(2)

    # Earning section
    # past earnings from alpha vintage API
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            df_stock_annualEPS = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            df_stock_qtrEPS = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    df_stock_qtrEPS[f'{col}'] = df_stock_qtrEPS[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue




    # # forecast 1 qtr earnings from alpha vantage API
    # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        forecast_earanings_df = pd.DataFrame(
            columns=my_list[0]
            ,data=my_list[1::]
            )
        
        latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)


    # forecast 1 year earnings from nasdaq webscrapping




    # Consolidated section
    df_stock_consolidate = df_stock_base.head(window_days)


    df_stock_consolidate_date = df_stock_consolidate.index
    for i in df_stock_consolidate_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_df_stock_qtrEPS = df_stock_qtrEPS[df_stock_qtrEPS['reportedDate']<= i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_df_stock_qtrEPS.head(4) 
        past_3_qtrs_EPS = filtered_df_stock_qtrEPS.head(3)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        df_stock_consolidate.loc[i, f"{symbol}_EPS_TTM"] = EPS_TTM

        if i == max(df_stock_consolidate.index):
            EPS_latest_projected = latest_projected_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        df_stock_consolidate[f"{symbol}_EPS_latest_projected"] = EPS_latest_projected
    
    # stock's stats
    df_stock_consolidate[f"{symbol}_PE_TTM"] = (df_stock_consolidate[symbol] / df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_avg"] = df_stock_consolidate[f"{symbol}_PE_TTM"].mean().round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_std"] = np.std(df_stock_consolidate[f"{symbol}_PE_TTM"]).round(2)
    df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_avg"] + df_stock_consolidate[f"{symbol}_PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_avg"] - df_stock_consolidate[f"{symbol}_PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    df_stock_consolidate[f"{symbol}_relative_valuation_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] * df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    df_stock_consolidate[f"{symbol}_relative_valuation_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] * df_stock_consolidate[f"{symbol}_EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    df_stock_consolidate[f"{symbol}_relative_valuation_median"] = (np.median([df_stock_consolidate[f"{symbol}_relative_valuation_+"], df_stock_consolidate[f"{symbol}_relative_valuation_-"]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    df_stock_consolidate[f"{symbol}_relative_valuation_projected_+"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_+"] * df_stock_consolidate[f"{symbol}_EPS_latest_projected"]).round(2) # 这个是relative valuation的价格上限
    df_stock_consolidate[f"{symbol}_relative_valuation_projected_-"] = (df_stock_consolidate[f"{symbol}_PE_TTM_volatility_-"] * df_stock_consolidate[f"{symbol}_EPS_latest_projected"]).round(2) # 这个是relative valuation的价格下限
    df_stock_consolidate[f"{symbol}_relative_valuation_projected_median"] = (np.median([df_stock_consolidate[f"{symbol}_relative_valuation_projected_+"], df_stock_consolidate[f"{symbol}_relative_valuation_projected_-"]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    df_stock_consolidate[f"{symbol}_{window_days}_price_min"] = df_stock_consolidate[symbol].min().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_max"] = df_stock_consolidate[symbol].max().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_avg"] = df_stock_consolidate[symbol].mean().round(2)
    df_stock_consolidate[f"{symbol}_{window_days}_price_std"] = np.std(df_stock_consolidate[symbol]).round(2)



    conditions = [
    (df_stock_consolidate[f"{symbol}"] < df_stock_consolidate[f"{symbol}_relative_valuation_-"]),
    (df_stock_consolidate[f"{symbol}"] > df_stock_consolidate[f"{symbol}_relative_valuation_+"]),
    ((df_stock_consolidate[f"{symbol}"] >= df_stock_consolidate[f"{symbol}_relative_valuation_-"]) & (df_stock_consolidate[f"{symbol}"] <= df_stock_consolidate[f"{symbol}_relative_valuation_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    df_stock_consolidate[f"{symbol}_curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        df_stock_consolidate.loc[condition, f"{symbol}_price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    f"{symbol}"
    # ,f"{symbol}_EPS_TTM"
    # ,f"{symbol}_EPS_latest_projected"
    ,f"{symbol}_PE_TTM"
    ,f"{symbol}_PE_TTM_avg"
    ,f"{symbol}_relative_valuation_+"
    ,f"{symbol}_relative_valuation_-"
    ,f"{symbol}_relative_valuation_median"
    ,f"{symbol}_relative_valuation_projected_+"
    ,f"{symbol}_relative_valuation_projected_-"
    ,f"{symbol}_relative_valuation_projected_median"
    ,f"{symbol}_price_valuation_assessment"
    ]

    ticker_dict[f'{symbol}'] = df_stock_consolidate[selected_cols]

  symbol                                name  reportDate fiscalDateEnding  \
0    TSM  Taiwan Semiconductor Manufacturing  2024-05-27       2024-03-31   
1    TSM  Taiwan Semiconductor Manufacturing  2024-07-18       2024-06-30   
2    TSM  Taiwan Semiconductor Manufacturing  2024-10-17       2024-09-30   
3    TSM  Taiwan Semiconductor Manufacturing  2025-01-16       2024-12-31   
4    TSM  Taiwan Semiconductor Manufacturing  2025-04-16       2025-03-31   

  estimate currency  
0     1.32      TWD  
1     1.36      TWD  
2               TWD  
3               TWD  
4               TWD  
  symbol                       name  reportDate fiscalDateEnding estimate  \
0   BABA  Alibaba Group Holding Ltd  2024-09-27       2024-06-30     2.05   
1   BABA  Alibaba Group Holding Ltd  2024-11-14       2024-09-30            
2   BABA  Alibaba Group Holding Ltd  2025-02-05       2024-12-31            

  currency  
0      USD  
1      USD  
2      USD  
  symbol          name  reportDate fiscalDat

In [67]:
ticker_dict['TSM']

KeyError: 'TSM'

In [68]:
ticker_dict['TSM']

KeyError: 'TSM'

In [69]:
ticker_dict['SMFG']

AttributeError: 'Index' object has no attribute '_format_flat'

             SMFG  SMFG_PE_TTM  SMFG_PE_TTM_avg  SMFG_relative_valuation_+  \
2024-05-16  12.02        11.90            12.59                      13.43   
2024-05-15  12.27        12.15            12.59                      13.43   
2024-05-14  11.81        13.42            12.59                      11.70   
2024-05-13  11.66        13.25            12.59                      11.70   
2024-05-10  11.60        13.18            12.59                      11.70   
2024-05-09  11.51        13.08            12.59                      11.70   
2024-05-08  11.44        13.00            12.59                      11.70   
2024-05-07  11.54        13.11            12.59                      11.70   
2024-05-06  11.61        13.19            12.59                      11.70   
2024-05-03  11.56        13.14            12.59                      11.70   
2024-05-02  11.44        13.00            12.59                      11.70   
2024-05-01  11.36        12.91            12.59                 

In [70]:
ticker_dict['MUFG']

AttributeError: 'Index' object has no attribute '_format_flat'

             MUFG  MUFG_PE_TTM  MUFG_PE_TTM_avg  MUFG_relative_valuation_+  \
2024-05-16   9.76        11.39             8.31                       7.63   
2024-05-15  10.30        12.02             8.31                       7.63   
2024-05-14  10.41         8.48             8.31                      10.92   
2024-05-13  10.28         8.38             8.31                      10.92   
2024-05-10  10.31         8.40             8.31                      10.92   
2024-05-09  10.29         8.39             8.31                      10.92   
2024-05-08  10.11         8.24             8.31                      10.92   
2024-05-07  10.16         8.28             8.31                      10.92   
2024-05-06  10.21         8.32             8.31                      10.92   
2024-05-03  10.14         8.26             8.31                      10.92   
2024-05-02  10.02         8.17             8.31                      10.92   
2024-05-01   9.91         8.08             8.31                 